In [1]:
import pandas as pd
# import zipfile
# with zipfile.ZipFile('D:\Download\Датасеты.zip', 'r') as zip_ref:
#     zip_ref.extractall('D:\Download')

In [42]:
attend = pd.read_csv('D:/Download/Датасеты/attend.csv')
attend = attend.rename(columns={"уникальный номер занятия": "numzan", "уникальный номер группы": "numgroup", "уникальный номер участника": "numclient",
                               "направление 2": "type2", "напраление 3": "type3", "онлайн/офлайн": "online", "дата занятия": "datezan",
                               "время начала занятия": "timebegin", "время окончания занятия": "timeend"})
attend.head()

,numzan,numgroup,numclient,type2,type3,online,datezan,timebegin,timeend
0,401346550,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
1,401346550,801346550,101385462,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
2,401346550,801346550,101421897,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
3,401346550,801346550,101354499,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
4,401346550,801346550,101421312,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00


In [43]:
attend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5901274 entries, 0 to 5901273
Data columns (total 9 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   numzan     int64 
 1   numgroup   int64 
 2   numclient  int64 
 3   type2      object
 4   type3      object
 5   online     object
 6   datezan    object
 7   timebegin  object
 8   timeend    object
dtypes: int64(3), object(6)
memory usage: 405.2+ MB


In [47]:
groups = pd.read_csv('D:/Download/Датасеты/groups.csv')
groups = groups.rename(columns={"уникальный номер": "numgroup", "направление 1": "type1", "направление 2": "type2", "направление 3": "type3", 
                       "адрес площадки": "address_plo", "округ площадки": "okrug", "район площадки": "region", "расписание в активных периодах": "active_schedule",
                      "расписание в закрытых периодах": "end_schedule", "расписание в плановом периоде": "plan_schedule"})
groups.head()

,numgroup,type1,type2,type3,address_plo,okrug,region,active_schedule,end_schedule,plan_schedule
0,801357270,Физическая активность,ОФП,ОФП,"город Москва, Саратовская улица, дом 16, корпус 2",Юго-Восточный административный округ,муниципальный округ Текстильщики,NaN,"c 01.01.2023 по 31.03.2023, Пн., Ср. 19:10-20:...",NaN
1,801356857,Физическая активность,ОФП,ОФП,"город Москва, Подольская улица, дом 5",Юго-Восточный административный округ,муниципальный округ Марьино,NaN,"c 09.01.2023 по 31.03.2023, Вт., Чт. 10:00-11:...",NaN
2,801351684,Физическая активность,ОФП,ОФП,"г. Москва, Базовская улица, дом 15, строение 1...","Северный административный округ, Северный адми...","муниципальный округ Западное Дегунино, муницип...",NaN,"c 09.01.2023 по 31.03.2023, Вт., Чт. 19:00-20:...",NaN
3,801353683,Физическая активность,ОФП,ОФП,"город Москва, улица Обручева, дом 28А, город М...","Юго-Западный административный округ, Юго-Запад...","муниципальный округ Обручевский, муниципальный...",NaN,"c 09.01.2023 по 31.03.2023, Пн., Ср. 13:30-14:...",NaN
4,801352164,Физическая активность,ОФП,ОФП,"город Москва, Воронцовский парк, дом 3, город ...","Юго-Западный административный округ, Юго-Запад...","муниципальный округ Обручевский, муниципальный...",NaN,"c 10.01.2023 по 28.02.2023, Вт., Пт. 12:00-13:...",NaN


In [86]:
groups.type3.unique()

array(['ОФП', 'Скандинавская ходьба', 'Фитнес', 'Бадминтон', 'Аэробика',
       'Лечебная физкультура', 'Физкультурно-оздоровительные занятия',
       'Танцевальная физкультура', 'Гимнастика', 'Дыхательная гимнастика',
       'Цигун', 'Адаптивная и тонизирующая гимнастика',
       'Дыхательная гимнастика по системе Стрельниковой',
       'Оздоровительная гимнастика', 'Суставная гимнастика',
       'Рукоделие и творчество', 'Бальные танцы',
       'Латиноамериканские танцы', 'Клубные танцы', 'Танцы для всех',
       'Пение', 'Хоровое пение', 'ИЗО', 'Акварельная живопись',
       'Различные техники рисования', 'Курсы компьютерной грамотности',
       'Английский язык', 'Английский язык для начинающих',
       'Здорово жить', 'Шахматы и шашки', 'Настольный теннис',
       'Мини-футбол', 'Футбол', 'История, культура Москвы',
       'Немецкий язык', 'Осваиваем мобильные устройства',
       'Плетение из бумаги, квиллинг, оригами', 'Волейбол', 'Пилатес',
       'Здоровая спина', 'Бодибалет', 

In [48]:
users = pd.read_csv('D:/Download/Датасеты/users.csv')
users = users.rename(columns={"уникальный номер": "numclient", "дата создание личного дела": "datereg", "пол": "sex", "дата рождения": "datebirth", 
                       "адрес проживания": "address_live"})
users.head()

,numclient,datereg,sex,datebirth,address_live
0,101391104,2019-02-26 15:52:09.000,Женщина,1959-09-10,"город москва, константинова, дом 30"
1,101437383,2022-09-14 13:46:41.079,Женщина,1958-06-29,"г. москва, большая серпуховская улица, дом 40,..."
2,101351338,2018-03-19 18:34:58.000,Женщина,1956-08-28,"город москва, летчика грицевца, дом 10"
3,101354578,2018-03-22 13:19:42.000,Женщина,1939-05-19,"город москва, лобачевского, дом 76"
4,101411997,2019-12-17 12:29:25.481,Женщина,1951-03-02,"г. москва, улица вертолётчиков, дом 13"


In [62]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52334 entries, 0 to 52333
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   numclient     52334 non-null  int64 
 1   datereg       52334 non-null  object
 2   sex           52334 non-null  object
 3   datebirth     52334 non-null  object
 4   address_live  52334 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.0+ MB


In [77]:
users.nunique()

numclient       52334
datereg         52270
sex                 2
datebirth       11013
address_live    33337
dtype: int64

In [80]:
users.isnull().value_counts()

numclient  datereg  sex    datebirth  address_live
False      False    False  False      False           52334
dtype: int64

In [56]:
plan = groups.merge(attend, how='outer', on='numgroup')
plan.head()

,numgroup,type1,type2_x,type3_x,address_plo,okrug,region,active_schedule,end_schedule,plan_schedule,numzan,numclient,type2_y,type3_y,online,datezan,timebegin,timeend
0,801357270,Физическая активность,ОФП,ОФП,"город Москва, Саратовская улица, дом 16, корпус 2",Юго-Восточный административный округ,муниципальный округ Текстильщики,NaN,"c 01.01.2023 по 31.03.2023, Пн., Ср. 19:10-20:...",NaN,401429211.0,101367434.0,ОФП,ОФП,Нет,2022-04-20,19:10:00,20:10:00
1,801357270,Физическая активность,ОФП,ОФП,"город Москва, Саратовская улица, дом 16, корпус 2",Юго-Восточный административный округ,муниципальный округ Текстильщики,NaN,"c 01.01.2023 по 31.03.2023, Пн., Ср. 19:10-20:...",NaN,401429211.0,101429822.0,ОФП,ОФП,Нет,2022-04-20,19:10:00,20:10:00
2,801357270,Физическая активность,ОФП,ОФП,"город Москва, Саратовская улица, дом 16, корпус 2",Юго-Восточный административный округ,муниципальный округ Текстильщики,NaN,"c 01.01.2023 по 31.03.2023, Пн., Ср. 19:10-20:...",NaN,401429211.0,101354951.0,ОФП,ОФП,Нет,2022-04-20,19:10:00,20:10:00
3,801357270,Физическая активность,ОФП,ОФП,"город Москва, Саратовская улица, дом 16, корпус 2",Юго-Восточный административный округ,муниципальный округ Текстильщики,NaN,"c 01.01.2023 по 31.03.2023, Пн., Ср. 19:10-20:...",NaN,401429211.0,101414173.0,ОФП,ОФП,Нет,2022-04-20,19:10:00,20:10:00
4,801357270,Физическая активность,ОФП,ОФП,"город Москва, Саратовская улица, дом 16, корпус 2",Юго-Восточный административный округ,муниципальный округ Текстильщики,NaN,"c 01.01.2023 по 31.03.2023, Пн., Ср. 19:10-20:...",NaN,401429231.0,101367434.0,ОФП,ОФП,Нет,2022-04-25,19:10:00,20:10:00


In [63]:
plan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5903742 entries, 0 to 5903741
Data columns (total 18 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   numgroup         int64 
 1   type1            object
 2   type2_x          object
 3   type3_x          object
 4   address_plo      object
 5   okrug            object
 6   region           object
 7   active_schedule  object
 8   end_schedule     object
 9   plan_schedule    object
 10  numzan           Int64 
 11  numclient        Int64 
 12  type2_y          object
 13  type3_y          object
 14  online           object
 15  datezan          object
 16  timebegin        object
 17  timeend          object
dtypes: Int64(2), int64(1), object(15)
memory usage: 867.1+ MB


In [70]:
plan.numclient.isnull().value_counts()

False    5901274
True        2468
Name: numclient, dtype: int64

In [73]:
plan.type2_y.isnull().value_counts()

False    5901274
True        2468
Name: type2_y, dtype: int64

In [71]:
plan.numzan.nunique()

647132

In [72]:
plan.numclient.nunique()

50906

In [75]:
# plan = plan.astype({"numzan": "int64"})
# plan = plan.astype({"numclient": "int64"})
# plan.head()

In [74]:
# df = plan.join(users.set_index('numclient'), on='numclient')
# df.head()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5902702 entries, 0 to 52333
Data columns (total 13 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   уникальный номер            int64  
 1   дата создание личного дела  object 
 2   пол                         object 
 3   дата рождения               object 
 4   адрес проживания            object 
 5   уникальный номер занятия    float64
 6   уникальный номер группы     float64
 7   направление 2               object 
 8   направление 3               object 
 9   онлайн/офлайн               object 
 10  дата занятия                object 
 11  время начала занятия        object 
 12  время окончания занятия     object 
dtypes: float64(2), int64(1), object(10)
memory usage: 630.5+ MB


In [76]:
# df['уникальный номер занятия'] = pd.to_numeric(df['уникальный номер занятия'], errors='coerce').astype(int)
# df['уникальный номер группы'] = pd.to_numeric(df['уникальный номер группы'], errors='coerce').astype(int)
# df = df.rename(columns={'уникальный номер': 'уникальный номер участника', 'направление 2': 'направление2', 'направление 3': 'направление3'})
# df = df[['уникальный номер группы', 'уникальный номер участника', 'уникальный номер занятия', 'дата создание личного дела', 'пол', 'дата рождения', 
#         'адрес проживания', 'направление 2', 'направление 3', 'онлайн/офлайн', 'дата занятия', 'время начала занятия', 'время окончания занятия']]
# df.head()

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5902702 entries, 0 to 52333
Data columns (total 13 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   уникальный номер группы     int32 
 1   уникальный номер участника  int64 
 2   уникальный номер занятия    int32 
 3   дата создание личного дела  object
 4   пол                         object
 5   дата рождения               object
 6   адрес проживания            object
 7   направление 2               object
 8   направление 3               object
 9   онлайн/офлайн               object
 10  дата занятия                object
 11  время начала занятия        object
 12  время окончания занятия     object
dtypes: int32(2), int64(1), object(10)
memory usage: 585.4+ MB


In [38]:
groups['уникальный номер'].isin(df["уникальный номер группы"]).value_counts()

True     24535
False     2468
Name: уникальный номер, dtype: int64

In [81]:
di = pd.read_excel('D:/Download/Датасеты/dict.xlsx')
di.head()

,Разметка: Для ума/ Для души / Для тела,id_level1,level1,id_level2,level2,id_level3,leve3,d_level1,d_level2,d_level3
0,Для ума,649,Игры,651,Интеллектуальные игры,1042,Иные интеллектуальные игры,"Занятия по изучению правил игр, основанных на ...",NaN,NaN
1,Для ума,649,Игры,651,Интеллектуальные игры,1040,Викторины,"Занятия по изучению правил игры, которая заклю...",NaN,NaN
2,Для ума,649,Игры,651,Интеллектуальные игры,1041,Квест,Занятия по изучению способов решений головолом...,NaN,NaN
3,Для ума,649,Игры,651,Интеллектуальные игры,1043,Брейн-ринг,"Занятия по изучению правил игры, в которой поб...",NaN,NaN
4,Для ума,649,Игры,650,Настольные игры,323,Современные настольные игры,"Занятия по изучению правил современных игр, ра...",NaN,NaN


In [84]:
di.d_level1.unique()

array(['Занятия по изучению правил игр, основанных на применении интеллекта и эрудиции, а также участие в них.',
       'Занятия по изучению правил игры, которая заключается в ответах на устные или письменные вопросы из различных областей знания, и участие в ней.',
       'Занятия по изучению способов решений головоломок за определенное время в формате интерактивных игр.',
       'Занятия по изучению правил игры, в которой побеждает самый эрудированный человек или команда, быстрее всех отвечающая на вопросы ведущего, а также участие в ней.',
       'Занятия по изучению правил современных игр, развивающих способность к быстрому принятию решений, позволяющих сохранить ясность ума и повышающих концентрацию внимания, а также участие в них.',
       'Занятия по изучению правил настольных игр (шахматы, шашки, русское лото и др.), а также участие в них с целью полезного времяпрепровождения и развития когнитивных способностей.',
       'Занятия по изучению правил игры в лото, развивающей навык